# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

##### Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [771]:
def priest(sex, age, resp, oxygen, HR, SBP, temp_c, alert, inspired_oxygen, perf_status): 
    
    """(str, str, int, float, int, int, float, str, str, str) -> float
    This function uses the logic from hhttps://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence to compute the PRIEST  score and ultimate 30-day 
    probability of adverse outcome.
    This function accounts for any combination of upper or lower case for any of the string inputs
    If any of the inputs are invalid the code will raise a ValueError
    
    >>> priest('FEMALE', 51, 18, 0.99, 65, 115, 38.1, 'Alert', 'Air', 'Limited strenuous activity, can do light activity')    
    0.03
    
    >>>  
    priest(5, 51, 18, 0.99, 65, 115, 38.1, 'Alert', 'Air', 'Limited strenuous activity, can do light activity')    
    ValueError: Sex must a string. You gave me a <class 'int'>
    
    >>>  
    priest('female', 51, 18, 0.99, 65, 115, 38.1, 5, 'Air', 'Limited strenuous activity, can do light activity')        
    alert must a string. You gave me a <class 'int'>
    
    >>>  
    priest('Male', 87, 22, 0.92, 65, 180, 37.1, 'Confused or not alert', 'Air', 'Limited strenuous activity, can do light activity')  
    0.46
    """
    
    priest_score = 0
    output = 0
    
    if type(sex) != str:
        raise ValueError('Sex must a string. You gave me a {}'.format(type(sex)))
    elif sex.lower() == "female":
        sex_points = 0
    elif sex.lower() == "male":
        sex_points = 1
     
    if type(age) != int:
        raise ValueError('Age must an integer. You gave me a {}'.format(type(age)))  
    elif age >= 16 and age <= 49:
        age_points = 0 
    elif age >=50 and age <= 65:
        age_points = 2
    elif age >= 66 and age <= 80:
        age_points = 3
    elif age > 80:
        age_points = 4
    
    if type(resp) != int:
        raise ValueError('resp must an integer. You gave me a {}'.format(type(resp)))       
    elif resp < 9 or resp > 24:
        resp_points = 3
    elif resp >=9 and resp <= 11:
        resp_points = 1
    elif resp >=12 and resp <=20:
        resp_points = 0
    elif resp >=21 and resp <= 24:
        resp_points = 2
  
    if type(oxygen) != float:
        raise ValueError('oxygen must an float. You gave me a {}'.format(type(oxygen))) 
    elif oxygen > 0.95:
        oxygen_points = 0
    elif oxygen == 0.94 or oxygen == 0.95:
        oxygen_points = 1
    elif oxygen == 0.93 or oxygen == 0.92:
        oxygen_points = 2
    elif oxygen < 0.92:
        oxygen_points = 3
      
    if type(HR) != int:
        raise ValueError('HR must an integer. You gave me a {}'.format(type(HR)))  
    elif HR < 41 or HR > 130: 
        HR_points = 3
    elif HR >= 41 and HR <= 50:
        HR_points = 1
    elif HR >=51 and HR <= 90:
        HR_points = 0
    elif HR >=91 and HR <= 110:
        HR_points = 1
    elif HR >= 111 and HR <= 130:
        HR_points = 2 
        
    if type(SBP) != int:
        raise ValueError('SBP must an integer. You gave me a {}'.format(type(SBP)))          
    elif SBP < 91 or SBP > 219:
        SBP_points = 3
    elif SBP >=91 and SBP <= 100:
        SBP_points = 2
    elif SBP >= 101 and SBP <= 110:
        SBP_points =1
    elif SBP >=111 and SBP <= 219:
        SBP_points = 0
        
    if type(temp_c) != float:
        raise ValueError('temp_c must an float. You gave me a {}'.format(type(temp_c)))          
    elif temp_c < 35.1:
        temp_points =  3
    elif (temp_c >= 35.1 and temp_c <= 36.0) or (temp_c >= 38.1 and temp_c <= 39.0):
        temp_points = 1
    elif temp_c >=36.1 and temp_c <=38.0:
        temp_points = 0
    elif temp_c > 39.0:
        temp_points = 2
    
    if type(alert) != str:
        raise ValueError('alert must a string. You gave me a {}'.format(type(alert)))
    elif alert.lower() == "alert":
        alert_points = 0
    elif alert.lower() == "confused or not alert":
        alert_points = 3
    
    if type(inspired_oxygen) != str:
        raise ValueError('inspired_oxygen must a string. You gave me a {}'.format(type(inspired_oxygen)))    
    elif inspired_oxygen.lower() == "air":
        inspired_oxygen_points = 0
    elif inspired_oxygen.lower() == "supplemental oxygen":
        inspired_oxygen_points = 2
    
    if type(perf_status) != str:
        raise ValueError('perf_status must a string. You gave me a {}'.format(type(perf_status)))   
    elif perf_status.lower() == "unrestricted normal activity":
        perf_status_points = 0
    elif  perf_status.lower() == "limited strenuous activity, can do light activity":
        perf_status_points = 1
    elif  perf_status.lower() == "limited activity, can self-care":
        perf_status_points = 2
    elif  perf_status.lower() == "limited self-care":
        perf_status_points = 3   
    elif  perf_status.lower() == "bed/chair bound, no self-care":
        perf_status_points = 4
        
    priest_score = sex_points + age_points + resp_points + oxygen_points + HR_points +SBP_points + temp_points + alert_points + inspired_oxygen_points + perf_status_points
    
    if priest_score == 0 or priest_score == 1:
        output = .01
    if priest_score == 2 or priest_score == 3:
        output = .02
    if priest_score == 4:
        output = .03
    if priest_score == 5:
        output = .09
    if priest_score ==6:
         output = .15
    if priest_score ==7:
         output = .18  
    if priest_score ==8:
         output = .22
    if priest_score ==9:
         output = .26
    if priest_score ==10:
         output = .29
    if priest_score ==11:
         output = .34
    if priest_score ==12:
         output = .38
    if priest_score ==13:
         output = .46
    if priest_score ==14:
         output = .47
    if priest_score ==15:
         output = .49
    if priest_score ==16:
         output = .55
    if priest_score >= 17 and priest_score <= 25:
         output = .59
    if priest_score > 26:
        output = .99
    
  
    return output
    

In [772]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('FEMALE', 51, 18, 0.99, 65, 115, 38.1, 'Alert', 'Air', 'Limited strenuous activity, can do light activity')    
Expecting:
    0.03
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [773]:
import requests
import json

def find_hospital(age, sex, risk):
    """(int, str, float)->str
    Use the professor's REST web service to lookup and return the
    name of the hospital where this person should be sent.
    
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    
    >>> find_hospital(50, 'male', 0.5)
    'Emory Dunwoody Medical Center'
    
    >>> find_hospital(95, 'female', 0.9)
    'Wesley Woods Geriatric Hospital'
    
    >>> find_hospital(23, 'female', 0.3)
    'Select Specialty Hospital - Northeast Atlanta'
    
    >>> find_hospital(5, 'male', 0.7)
    'Hughes Spalding Childrens Hospital'
    """
    
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    
    try:
        r = requests.get(url)
        info = r.json()
        hospital = info.get('hospital')
    except:
        hospital = None
        raise ValueError(f'Unable to find hospital for inputs: {age} {sex} {risk}')
    
    return hospital

In [774]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(50, 'male', 0.5)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(95, 'female', 0.9)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(23, 'female', 0.3)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(5, 'male', 0.7)
Expecting:
    'Hughes Spalding Childrens Hospital'
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [744]:
from bs4 import BeautifulSoup
import requests

def get_address(hospital): 
    """(str)->str
    This function takes the name of a hospital and outputs the address from the Georgia Hospitals Directory website 
    
    >>> get_address('CHARTER BEHAVIORAL HEALTH SYSTEM OF ATLANTA')
    '811 JUNIPER STREET NE'
    
    >>> get_address('SELECT SPECIALTY HOSPITAL - ATLANTA')
    '550 PEACHTREE STREET, NE'
    
    >>> get_address('PRIDE MEDICAL')
    '3280 HOWELL MILL ROAD NW'
    
    
    """
    r= requests.get('https://www.officialusa.com/stateguides/health/hospitals/georgia.html')

    output = {}
    
    soup=BeautifulSoup(r.text)
    for table in soup.find_all(('table',{'id': 'myTable'})):
        #print(table.text)
        headers=[]
        for header in table.find_all('th'): # ('tr',{'style': 'display: none'})):
            headers.append(header.text)
        if headers ==['City', 'Hospital Name', 'TYPE', 'ADDRESS', 'WEBSITE', 'BEDS', 'NAICS_DESC']:
            for row in table.find_all('tr'):
                data = [x.text for x in list(row.find_all('td'))]
                if len(data) != 0:
                
                    city = data[0]
                    hospital_name = data[1]
                    hospital_type = data[2]
                    hospital_address = data[3]
                    hospital_website = data[4]
                    beds = data[5]
                    NAICS_DESC =data [6]
                    try:
                        if city == "ATLANTA" and hospital_name == hospital:
                            output = hospital_address
                           
                    except  KeyError:
                            print('Unable to find address for hospital input:{hospital}')
            return output

In [746]:
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address('CHARTER BEHAVIORAL HEALTH SYSTEM OF ATLANTA')
Expecting:
    '811 JUNIPER STREET NE'
ok
Trying:
    get_address('SELECT SPECIALTY HOSPITAL - ATLANTA')
Expecting:
    '550 PEACHTREE STREET, NE'
ok
Trying:
    get_address('PRIDE MEDICAL')
Expecting:
    '3280 HOWELL MILL ROAD NW'
ok


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.

In [747]:
def process_people(file):
     """(psv)->dict 
    This function takes a file that includes a list of people, their sex, age, respiratory rate, O2 sat, heart rate, systolilc blood pressure, temperature (in Celsius), 
    alertness, inspired oxygen, and activity status and calculates their PRIEST risk score, then determines which hospital they should be transferred to and the address
    of that hospital.  This is all output into a final dictionary of the patient's ID number, parameters that went into the risk score, the calculated risk score, hospital and address
    """
    import csv
    with open(file) as f:
    
        reader = csv.reader(f, delimiter='|')
        header = []
        header = next(reader)
        rows = []
        csv_list = []
        global final_encounters
        final_encounters = {} 
        for row in reader:
            rows.append(row)
        for item in rows:
            patient= item[0]
            sex= item[1]
            age= int(item[2])
            resp= int(item[3])
            oxygen= float(item[4])
            HR= int(item[5])
            SBP = int(item[6])
            temp_c= float(item [7])
            alert= item[8]
            inspired_oxygen= item[9]
            perf_status= item[10]
            final_encounters[patient]= [sex, age, resp, oxygen, HR, SBP, temp_c, alert, inspired_oxygen, perf_status]
            #print(encounters)
            for v in final_encounters:
                risk = priest(sex, age, resp, oxygen, HR, SBP, temp_c, alert, inspired_oxygen, perf_status)
            location = find_hospital(age, sex, risk) 
            location= location.upper()         
            location_address = get_address(location)   
            final_encounters[patient].append(risk)
            final_encounters[patient].append(location)
            final_encounters[patient].append(location_address) 
        
        return final_encounters
        

In [748]:
process_people('/home/jupyter-msimkins7/hds5210-2022/people.psv') 

{'E9559': ['FEMALE',
  40,
  24,
  0.96,
  105,
  115,
  34.9,
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  0.15,
  'SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA',
  '1821 CLIFTON ROAD NE'],
 'E9385': ['Female',
  51,
  19,
  0.99,
  90,
  91,
  38.1,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  0.18,
  'SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA',
  '1821 CLIFTON ROAD NE'],
 'E3067': ['female',
  40,
  29,
  0.96,
  105,
  95,
  38.1,
  'ALERT',
  'air',
  'unrestricted normal activity',
  0.18,
  'SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA',
  '1821 CLIFTON ROAD NE'],
 'E9422': ['FEMALE',
  66,
  19,
  0.96,
  135,
  115,
  38.1,
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  0.34,
  'WESLEY WOODS GERIATRIC HOSPITAL',
  '1821 CLIFTON ROAD, NE'],
 'E8661': ['MALE',
  18,
  22,
  0.95,
  105,
  115,
  36.3,
  'Confused or Not Alert',
  'air',
  'unrestricted normal activity',
  0.22,
  'SOUTHWEST HOSPITAL AND MEDICAL CENTER',
  '50

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [751]:
import json
with open('/home/jupyter-msimkins7/hds5210-2022/people_results.json') as f:
    obj = json.load(f)
    print(len(obj))  


99


In [755]:
calc_risk =process_people('/home/jupyter-msimkins7/hds5210-2022/people.psv')
print(len(calc_risk))

99


In [765]:
def output(answer):
        for key, value in answer.items():
            print(key,':',value)   

In [767]:
patient_answer = str(output(calc_risk))
obj_answer = str(output(obj))

In [768]:
patient_answer==obj_answer

True

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [1]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main d88bc5b] Submitting the week 7 programming exercises
 2 files changed, 297 insertions(+), 8 deletions(-)
 create mode 100644 week07/week07_assignment_2.ipynb
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 3.66 KiB | 3.66 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:paulboal/hds5210-2022.git
   fc2200c..d88bc5b  main -> main
